In [54]:
import requests
from dotenv import dotenv_values

In [74]:
class API_Loader:
    def __init__(self, access_token, client_secret, client_id, refresh_token):
        self.access_token = 'd8e5b3cdabb20065ea428868b776d7badb6f9331'
        self.client_secret = client_secret
        self.client_id = client_id
        self.refresh_token = refresh_token

    def _get_headers(self):
        """

        :return:
        """
        return {'Authorization': f'Bearer {self.access_token}'}

    def refresh_access_token(self):
        """
        Rafraîchit le token d'accès à chaque appel.
        """
        url = "https://www.strava.com/api/v3/oauth/token"
        payload = {
            "client_id": self.client_id,
            "client_secret": self.client_secret,
            "grant_type": "refresh_token",
            "refresh_token": self.refresh_token
        }
        response = requests.post(url, data=payload)
        if response.status_code == 200:
            token_data = response.json()
            self.access_token = token_data.get("access_token")
            print("Jeton rafraîchi avec succès.")
            return True
        else:
            print("Erreur lors du rafraîchissement du token :", response.status_code, response.text)
            return False

    def get_athlete_data(self):
        """
        Récupère les informations de l'athlète connecté.
        Rafraîchit le token à chaque appel.
        """
        if not self.refresh_access_token():
            print("Impossible de rafraîchir le token.")
            return None

        url = "https://www.strava.com/api/v3/athlete"
        response = requests.get(url, headers=self._get_headers())

        if response.status_code == 200:
            return response.json()
        else:
            print("Erreur lors de la requête :", response.status_code, response.text)
            return None

    def get_recent_activities(self, after=1735691212):
        """
        Récupère la liste des activités récentes de l'athlète connecté.

        Args:
            page (int): Le numéro de page à récupérer (utile pour la pagination).
            per_page (int): Le nombre d'activités par page.

        Returns:
            list: Une liste de dicts contenant les informations de chaque activité.
        """
        # Rafraîchit le token à chaque appel
        if not self.refresh_access_token():
            print("Impossible de rafraîchir le token.")
            return None

        url = "https://www.strava.com/api/v3/athlete/activities"
        params = {
            'after': after
        }
        response = requests.get(url, headers=self._get_headers(), params=params)
        if response.status_code == 200:
            return response.json()
        else:
            print("Erreur lors de la récupération des activités :", response.status_code, response.text)
            return None



In [75]:
secrets = dotenv_values('.env')

In [76]:
api = API_Loader(
    access_token=None,
    client_secret=secrets['STRAVA_SECRET'],
    client_id=secrets['STRAVA_CLIENT_ID'],
    refresh_token=secrets['STRAVA_REFRESH_TOKEN']
)

In [77]:
api.get_recent_activities()

Jeton rafraîchi avec succès.
Erreur lors de la récupération des activités : 401 {"message":"Authorization Error","errors":[{"resource":"AccessToken","field":"activity:read_permission","code":"missing"}]}


In [61]:
api.get_athlete_data()

Jeton rafraîchi avec succès.


{'id': 86360204,
 'username': 'elancelle',
 'resource_state': 2,
 'firstname': 'Etienne',
 'lastname': 'Lancelle',
 'bio': '➡️ Semi-marathon de Paris (09/03)  \n10K (x4) : 54’54’’  \n21K (x3) : 1:59’00’’  \n42K : ?',
 'city': 'Boulogne-Billancourt',
 'state': None,
 'country': None,
 'sex': 'M',
 'premium': False,
 'summit': False,
 'created_at': '2021-05-31T15:22:52Z',
 'updated_at': '2025-02-02T15:21:04Z',
 'badge_type_id': 0,
 'weight': 65.0,
 'profile_medium': 'https://dgalywyr863hv.cloudfront.net/pictures/athletes/86360204/33884293/1/medium.jpg',
 'profile': 'https://dgalywyr863hv.cloudfront.net/pictures/athletes/86360204/33884293/1/large.jpg',
 'friend': None,
 'follower': None}

In [68]:
client_id = secrets['STRAVA_CLIENT_ID']
redirect_uri = "http://localhost:5000/callback"  # Assure-toi que cette URL est enregistrée dans ton application Strava
# Demande de tous les scopes. Selon la doc, ils doivent être séparés par des virgules.
scope = "profile:read_all,activity:read_all"

auth_url = (
    f"https://www.strava.com/oauth/authorize?"
    f"client_id={client_id}&"
    f"redirect_uri={redirect_uri}&"
    f"response_type=code&"
    f"approval_prompt=force&"
    f"scope={scope}"
)

print("Ouvre cette URL dans ton navigateur pour autoriser l'application :")
print(auth_url)


Ouvre cette URL dans ton navigateur pour autoriser l'application :
https://www.strava.com/oauth/authorize?client_id=127031&redirect_uri=http://localhost:5000/callback&response_type=code&approval_prompt=force&scope=profile:read_all,activity:read_all


In [72]:
url = "https://www.strava.com/api/v3/oauth/token"
data = {
    'client_id': '127031',
    'client_secret': '641f9f581fc9ce96833f6d863361f447fd2e2ac0',
    'grant_type': 'refresh_token',
    'refresh_token': '0125eaf0ccf6a09cb538eccfea341e9219206a2e'
}
response = requests.post(url, data=data, timeout=20).json()

In [73]:
response

{'token_type': 'Bearer',
 'access_token': 'd8e5b3cdabb20065ea428868b776d7badb6f9331',
 'expires_at': 1739645717,
 'expires_in': 20661,
 'refresh_token': '0125eaf0ccf6a09cb538eccfea341e9219206a2e'}

In [ ]:
https://www.strava.com/oauth/authorize?client_id=127031&redirect_uri=http://localhost:5000/callback&response_type=code&scope=activity:read_all

In [ ]:
http://localhost:5000/callback?state=&code=1f9bafa1ec95e60a1e6b84a88b190bd91d5c1c64&scope=read,activity:read_all

In [ ]:
https://www.strava.com/oauth/authorize?client_id=127031&redirect_uri=http://localhost:5000/callback&response_type=code&scope=activity:read_all&approval_prompt=force

In [ ]:
http://localhost:5000/callback?state=&code=5f7916556cf3954463e7079dda38f8466f31f8df&scope=read,activity:read_all

In [99]:
import requests

# Remplace par tes infos
CLIENT_ID = "127031"
CLIENT_SECRET = "641f9f581fc9ce96833f6d863361f447fd2e2ac0"
CODE = "2f45e57ede72d174e9dc2172ebcc6b28d8ad1f22"  # Remplace avec le code obtenu à l'étape 1

# Échange du code contre un token d'accès
url = "https://www.strava.com/oauth/token"
params = {
    "client_id": CLIENT_ID,
    "client_secret": CLIENT_SECRET,
    "code": CODE,
    "grant_type": "authorization_code",
}

response = requests.post(url, data=params)
tokens = response.json()

if "access_token" in tokens:
    print("✅ Jeton d'accès récupéré avec succès !")
    print("Access Token:", tokens["access_token"])
    print("Refresh Token:", tokens["refresh_token"])
    print("Expire dans:", tokens["expires_in"], "secondes")
else:
    print("❌ Erreur :", tokens)

✅ Jeton d'accès récupéré avec succès !
Access Token: 79f3c343cf1dd482f3843540649416ebb4bf70cd
Refresh Token: e4c6dab4ad00d26aaf19712d22a1006210ac3ef2
Expire dans: 8632 secondes


In [100]:
import requests

REFRESH_TOKEN = "e4c6dab4ad00d26aaf19712d22a1006210ac3ef2"  # À récupérer depuis l'étape 2

def refresh_token():
    url = "https://www.strava.com/oauth/token"
    params = {
        "client_id": CLIENT_ID,
        "client_secret": CLIENT_SECRET,
        "refresh_token": REFRESH_TOKEN,
        "grant_type": "refresh_token",
    }

    response = requests.post(url, data=params)
    tokens = response.json()

    if "access_token" in tokens:
        print("✅ Nouveau jeton d'accès récupéré !")
        print("Access Token:", tokens["access_token"])
        print("Refresh Token:", tokens["refresh_token"])
        return tokens["access_token"], tokens["refresh_token"]
    else:
        print("❌ Erreur :", tokens)
        return None, None

# Utilisation
access_token, refresh_token = refresh_token()

✅ Nouveau jeton d'accès récupéré !
Access Token: 79f3c343cf1dd482f3843540649416ebb4bf70cd
Refresh Token: e4c6dab4ad00d26aaf19712d22a1006210ac3ef2


In [101]:

ACCESS_TOKEN = "79f3c343cf1dd482f3843540649416ebb4bf70cd"

url = "https://www.strava.com/api/v3/athlete/activities"
headers = {"Authorization": f"Bearer {ACCESS_TOKEN}"}
params = {"per_page": 50}  # Nombre d'activités à récupérer

response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    activities = response.json()
    for i, act in enumerate(activities, 1):
        print(f"{i}. {act['name']} - {act['distance']}m")
else:
    print("❌ Erreur :", response.json())

1. 🗼 Lunch Run (🧡 J-22 & 🗼 J-57) - 4796.5m
2. 🗼 SL 100% plaisir sous le soleil (🧡 J-23 & 🗼 J-58) - 11450.7m
3. 🗼 Lunch Run (🧡 J-24 & 🗼 J-59) - 5570.1m
4. 🗼 Evening Run 🏃🏻‍♂️🏃🏻‍♀️ (🧡 J-25 & 🗼 J-60) - 6728.0m
5. 🗼 Un petit tour ! (🧡 J-26 & 🗼 J-61) - 3783.5m
6. 🗼 Mini-run : croquettes pour chat à aller chercher ! (🧡 J-27 & 🗼 J-62) - 2695.7m
7. 🗼 Domaine St-Cloud ⚜️ (🧡 J-28 & 🗼 J-63) - 10402.9m
8. 🗼 Evening Run (🧡 J-29 & 🗼 J-64) - 4097.2m
9. 🗼 La Spirale 🌀 🏃🏻‍♂️🏃🏻‍♀️ (🧡 J-30 & 🗼 J-65) - 16111.3m
10. 🗼 Afternoon Run (🧡 J-31 & 🗼 J-66) - 3934.9m
11. 🗼 Fractionné du déjeuner 🥵 (🧡 J-32 & 🗼 J-67) - 9992.9m
12. 🗼 EF 45’ 🧊 🏃🏻‍♂️🏃🏻‍♀️ (🧡 J-33 & 🗼 J-68) - 6366.2m
13. 🗼 Evening Run 🏃🏻‍♂️🏃🏻‍♀️ (🧡 J-34 & 🗼 J-69) - 6532.7m
14. 🗼 SL 1h40 ☀️ (🧡 J-35 & 🗼 J-70) - 15651.0m
15. 🗼 Night Run (🧡 J-36 & 🗼 J-71) - 4196.5m
16. 🚶🏻‍♂️ La balade sous le soleil ☀️ ft. Enora 🌸 - 6468.5m
17. 🗼 Evening Run (🧡 J-37 & 🗼 J-72) - 6068.1m
18. 🗼 Night Run (🧡 J-38 & 🗼 J-73) - 4138.6m
19. 🗼 Variations d’allures 🚀 (🧡 J-39 & 🗼 J-74

In [84]:
activities[3]

{'resource_state': 2,
 'athlete': {'id': 86360204, 'resource_state': 1},
 'name': '🗼 Evening Run 🏃🏻\u200d♂️🏃🏻\u200d♀️ (🧡 J-25 & 🗼 J-60)',
 'distance': 6728.0,
 'moving_time': 2420,
 'elapsed_time': 2445,
 'total_elevation_gain': 22.0,
 'type': 'Run',
 'sport_type': 'Run',
 'workout_type': None,
 'id': 13610646858,
 'start_date': '2025-02-12T18:01:31Z',
 'start_date_local': '2025-02-12T19:01:31Z',
 'timezone': '(GMT+01:00) Europe/Paris',
 'utc_offset': 3600.0,
 'location_city': None,
 'location_state': None,
 'location_country': None,
 'achievement_count': 3,
 'kudos_count': 20,
 'comment_count': 0,
 'athlete_count': 2,
 'photo_count': 0,
 'map': {'id': 'a13610646858',
  'summary_polyline': 'mo_iHq}tLIa@@S`@eB^kCLk@x@uBHu@^yAPyA`@{AJy@jAeEPkADKLIHAJDFEPc@D[DKDCPCvB}@p@_@n@QbAe@PEJBFA^Y^UfBu@B?x@[ZQLDHNTDHBb@r@Z~@Ab@Fn@bAnFJx@R`A@NK^?TNz@l@zEd@vFLhDAjACb@Ar@?z@HrBBpCG|CKbBE^Ov@_@xCc@lBcAhGYvBUlAU|@CX]rB_@lBqAhEER?PCLQf@GVa@nAm@dBMj@Qh@Yh@M\\mCrDq@p@kAxASPR[t@w@nBcCz@mANOPYRg@Nq@D?f@sAr@y

In [86]:

ACCESS_TOKEN = "79f3c343cf1dd482f3843540649416ebb4bf70cd"

url = "https://www.strava.com/api/v3/activities/13610646858"
headers = {"Authorization": f"Bearer {ACCESS_TOKEN}"}
params = {}  # Nombre d'activités à récupérer

response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    activities = response.json()
    for i, act in enumerate(activities, 1):
        print(f"{i}. {act['name']} - {act['distance']}m")
else:
    print("❌ Erreur :", response.json())

TypeError: string indices must be integers, not 'str'

In [89]:
act = response.json()

In [90]:
import pprint

In [91]:
pprint.pprint(act)

{'achievement_count': 3,
 'athlete': {'id': 86360204, 'resource_state': 1},
 'athlete_count': 2,
 'available_zones': [],
 'average_cadence': 87.3,
 'average_heartrate': 152.2,
 'average_speed': 2.78,
 'average_temp': 22,
 'average_watts': 250.4,
 'best_efforts': [{'achievements': [],
                   'activity': {'id': 13610646858,
                                'resource_state': 1,
                                'visibility': 'followers_only'},
                   'athlete': {'id': 86360204, 'resource_state': 1},
                   'distance': 400,
                   'elapsed_time': 130,
                   'end_index': 1826,
                   'id': 57566494690,
                   'moving_time': 130,
                   'name': '400m',
                   'pr_rank': None,
                   'resource_state': 2,
                   'start_date': '2025-02-12T18:01:31Z',
                   'start_date_local': '2025-02-12T19:01:31Z',
                   'start_index': 1696},
              

In [92]:
import requests

ACCESS_TOKEN = "79f3c343cf1dd482f3843540649416ebb4bf70cd"

url = "https://www.strava.com/api/v3/athlete"
headers = {"Authorization": f"Bearer {ACCESS_TOKEN}"}

response = requests.get(url, headers=headers)

if response.status_code == 200:
    print("✅ Ton access_token est valide !")
    print(response.json())  # Affiche les infos de ton compte
else:
    print("❌ Erreur :", response.json())  # Vérifie si ton scope est suffisant

✅ Ton access_token est valide !
{'id': 86360204, 'username': 'elancelle', 'resource_state': 2, 'firstname': 'Etienne', 'lastname': 'Lancelle', 'bio': '➡️ Semi-marathon de Paris (09/03)  \n10K (x4) : 54’54’’  \n21K (x3) : 1:59’00’’  \n42K : ?', 'city': 'Boulogne-Billancourt', 'state': None, 'country': None, 'sex': 'M', 'premium': False, 'summit': False, 'created_at': '2021-05-31T15:22:52Z', 'updated_at': '2025-02-02T15:21:04Z', 'badge_type_id': 0, 'weight': 65.0, 'profile_medium': 'https://dgalywyr863hv.cloudfront.net/pictures/athletes/86360204/33884293/1/medium.jpg', 'profile': 'https://dgalywyr863hv.cloudfront.net/pictures/athletes/86360204/33884293/1/large.jpg', 'friend': None, 'follower': None}


In [97]:
secrets.get('STRAVA_CLIENT_ID')
callback = 'http://localhost:5000/callback'

In [98]:
f"https://www.strava.com/oauth/authorize?client_id={secrets.get('STRAVA_CLIENT_ID')
}&redirect_uri={callback}&response_type=code&scope=activity:read_all&approval_prompt=force"

'https://www.strava.com/oauth/authorize?client_id=127031&redirect_uri=http://localhost:5000/callback&response_type=code&scope=activity:read_all&approval_prompt=force'

In [ ]:
http://localhost:5000/callback?state=&code=2f45e57ede72d174e9dc2172ebcc6b28d8ad1f22&scope=read,activity:read_all

In [102]:
import requests

ACCESS_TOKEN = secrets.get('STRAVA_ACCESS_TOKEN')

url = "https://www.strava.com/api/v3/athlete"
headers = {"Authorization": f"Bearer {ACCESS_TOKEN}"}

response = requests.get(url, headers=headers)

if response.status_code == 200:
    print("✅ Ton access_token est valide !")
    print(response.json())  # Affiche les infos de ton compte
else:
    print("❌ Erreur :", response.json())  # Vérifie si ton scope est suffisant

❌ Erreur : {'message': 'Authorization Error', 'errors': [{'resource': 'Athlete', 'field': 'access_token', 'code': 'invalid'}]}


In [103]:
f"https://www.strava.com/oauth/authorize?client_id={secrets.get('STRAVA_CLIENT_ID')}&redirect_uri={callback}&response_type=code&scope=activity:read_all&approval_prompt=force"

'https://www.strava.com/oauth/authorize?client_id=127031&redirect_uri=http://localhost:5000/callback&response_type=code&scope=activity:read_all&approval_prompt=force'

In [ ]:
http://localhost:5000/callback?state=&code=8866f64d415954334e9e998092a5a1e5f41dedbb&scope=read,activity:read_all

In [105]:
secrets

OrderedDict([('STRAVA_REFRESH_TOKEN',
              '0125eaf0ccf6a09cb538eccfea341e9219206a2e'),
             ('STRAVA_CLIENT_ID', '127031'),
             ('STRAVA_SECRET', '641f9f581fc9ce96833f6d863361f447fd2e2ac0')])

In [106]:
import requests

CLIENT_ID = secrets.get('STRAVA_CLIENT_ID')
CLIENT_SECRET = secrets.get('STRAVA_SECRET')
CODE = "8866f64d415954334e9e998092a5a1e5f41dedbb"

url = "https://www.strava.com/oauth/token"
params = {
    "client_id": CLIENT_ID,
    "client_secret": CLIENT_SECRET,
    "code": CODE,
    "grant_type": "authorization_code",
}

response = requests.post(url, data=params)
tokens = response.json()

if "access_token" in tokens:
    print("✅ Nouveau jeton d'accès récupéré !")
    print("Access Token:", tokens["access_token"])
    print("Scopes autorisés :", tokens)
else:
    print("❌ Erreur :", tokens)

✅ Nouveau jeton d'accès récupéré !
Access Token: 79f3c343cf1dd482f3843540649416ebb4bf70cd
Scopes autorisés : {'token_type': 'Bearer', 'expires_at': 1739647687, 'expires_in': 8364, 'refresh_token': 'e4c6dab4ad00d26aaf19712d22a1006210ac3ef2', 'access_token': '79f3c343cf1dd482f3843540649416ebb4bf70cd', 'athlete': {'id': 86360204, 'username': 'elancelle', 'resource_state': 2, 'firstname': 'Etienne', 'lastname': 'Lancelle', 'bio': '➡️ Semi-marathon de Paris (09/03)  \n10K (x4) : 54’54’’  \n21K (x3) : 1:59’00’’  \n42K : ?', 'city': 'Boulogne-Billancourt', 'state': None, 'country': None, 'sex': 'M', 'premium': False, 'summit': False, 'created_at': '2021-05-31T15:22:52Z', 'updated_at': '2025-02-02T15:21:04Z', 'badge_type_id': 0, 'weight': 65.0, 'profile_medium': 'https://dgalywyr863hv.cloudfront.net/pictures/athletes/86360204/33884293/1/medium.jpg', 'profile': 'https://dgalywyr863hv.cloudfront.net/pictures/athletes/86360204/33884293/1/large.jpg', 'friend': None, 'follower': None}}


In [ ]:
datetime.datetime.strftime('2025-01-01', '%Y-%m-%d')